In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

In [ ]:
# Downloading and unzipping the data
!wget -O baci.zip 'https://www.cepii.fr/DATA_DOWNLOAD/baci/data/BACI_HS96_V202501.zip'
!unzip baci.zip

In [ ]:
# Chapter classification as mentioned on the website (https://hts.usitc.gov/). This will help us aggregate products by chapter.
# Define the mapping of chapters to categories
categories = [
    (range(1, 6), "Live Animals; Animal Products"),
    (range(6, 15), "Vegetable Products"),
    (range(15, 16), "Animal or Vegetable Fats and Oils and Their Cleavage Products; Prepared Edible Fats; Animal or Vegetable Waxes"),
    (range(16, 25), "Prepared Foodstuffs; Beverages, Spirits, and Vinegar; Tobacco and Manufactured Tobacco Substitutes"),
    (range(25, 28), "Mineral Products"),
    (range(28, 39), "Products of the Chemical or Allied Industries"),
    (range(39, 41), "Plastics and Articles Thereof; Rubber and Articles Thereof"),
    (range(41, 44), "Raw Hides and Skins, Leather, Furskins and Articles Thereof; Saddlery and Harness; Travel Goods, Handbags and Similar Containers; Articles of Animal Gut (Other Than Silkworm Gut)"),
    (range(44, 47), "Wood and Articles of Wood; Wood Charcoal; Cork and Articles of Cork; Manufacturers of Straw, of Esparto or of Other Plaiting Materials; Basketware and Wickerwork"),
    (range(47, 50), "Pulp of Wood or of Other Fibrous Cellulosic Material; Waste and Scrap of Paper or Paperboard; Paper and Paperboard and Articles Thereof"),
    (range(50, 64), "Textile and Textile Articles"),
    (range(64, 68), "Footwear, Headgear, Umbrellas, Sun Umbrellas, Walking Sticks, Seatsticks, Whips, Riding-Crops and Parts Thereof; Prepared Feathers and Articles Made Therewith; Artificial Flowers; Articles of Human Hair"),
    (range(68, 71), "Articles of Stone, Plaster, Cement, Asbestos, Mica or Similar Materials; Ceramic Products; Glass and Glassware"),
    (range(71, 72), "Natural or Cultured Pearls, Precious or Semiprecious Stones, Precious Metals, Metals Clad With Precious Metal, and Articles Thereof; Imitation Jewelry; Coin"),
    (range(72, 84), "Base Metals and Articles of Base Metal"),
    (range(84, 86), "Machinery and Mechanical Appliances; Electrical Equipment; Parts Thereof; Sound Recorders and Reproducers, Television Image and Sound Recorders and Reproducers, and Parts and Accessories of Such Articles"),
    (range(86, 90), "Vehicles, Aircraft, Vessels and Associated Transport Equipment"),
    (range(90, 93), "Optical, Photographic, Cinematographic, Measuring, Checking, Precision, Medical or Surgical Instruments and Apparatus; Clocks and Watches; Musical Instruments; Parts and Accessories Thereof"),
    (range(93, 94), "Arms and Ammunition; Parts and Accessories Thereof"),
    (range(94, 97), "Miscellaneous Manufactured Articles"),
    (range(97, 98), "Works of Art, Collectors' Pieces and Antiques"),
    (range(98, 100), "Special Classification Provisions; Temporary Legislation; Temporary Modifications Proclaimed pursuant to Trade Agreements Legislation; Additional Import Restrictions Proclaimed Pursuant to Section 22 of the Agricultural Adjustment Act, As Amended"),
]

# Generate the data
data = []
for chapter_range, category in categories:
    for chapter in chapter_range:
        data.append((chapter, category))

# Create DataFrame
chapters_df = pd.DataFrame(data, columns = ["chapter", "chapter_description"])
chapters_df['chapter'] = chapters_df['chapter'].astype(str)

In [ ]:
# Merging all the datasets into a single one
# Define the directory containing the files
file_list = os.listdir('/content')
directory = "/content"

# Filter files that start with "BACI_HS96" and end with ".csv"
baci_files = [file for file in file_list if file.startswith("BACI_HS96") and file.endswith(".csv")]

# Create an empty DataFrame to store the combined data
df_dicts = {}

# Loop through each file and append the data
for file in sorted(baci_files):
    file_path = os.path.join(directory, file)  # Construct full file path
    try:
        df = pd.read_csv(file_path)  # Read CSV file
        df = df.rename(columns = {'t' : 'year', 'i' : 'exporter',
                           'j' : 'importer', 'k' : 'product_code',
                           'v' : 'value', 'q' : 'quantity'})

        # To homogenize for chapter mapping
        df['product_code'] = df["product_code"].astype(str).str.zfill(6)

        # Extracting chapter and removing leading zeroes for mapping
        df['chapter'] = df['product_code'].str[:2].astype(str).str.lstrip('0')
        df_agg = df.merge(chapters_df, on = 'chapter').groupby(by = ['year', 'exporter',
                                                                     'importer', 'chapter',
                                                                     'chapter_description'])['value'].sum().to_frame().reset_index()

        df_dicts[df_agg['year'].unique()[0]] = df_agg
        #print(f"processed: {df_agg['year'].unique()[0]}")  # Optional log
    except Exception as e:
        print(f"Error reading {file}: {e}")  # Handle potential read errors

In [ ]:
baci_df = pd.concat(df_dicts.values(), ignore_index = True)

baci_df.to_csv('BACI_combined.csv', index = False)

zip_filename = 'BACI_combined.zip'
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write('BACI_combined.csv')

#print(f"Zipped file saved as {zip_filename}")